In [1]:
import SymbolicSolver
import QuestionInference
import ImageGenerator
from gensim.models import KeyedVectors
import numpy as np
import pandas as pd
from pyprob import InferenceEngine

Cannot import dbm.gnu: No module named '_gdbm'


C:\Users\bprys\miniconda3\lib\site-packages\pyprob\util.py:327: UserWarning: Empirical distributions on disk may perform slow because GNU DBM is not available. Please install and configure gdbm library for Python for better speed.
  warnings.warn('Empirical distributions on disk may perform slow because GNU DBM is not available. Please install and configure gdbm library for Python for better speed.')
C:\Users\bprys\miniconda3\lib\site-packages\pl_bolts\utils\warnings.py:30: UserWarning: You want to use `wandb` which is not installed yet, install it with `pip install wandb`.
  stdout_func(
C:\Users\bprys\miniconda3\lib\site-packages\pl_bolts\utils\warnings.py:30: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
  stdout_func(


In [4]:
wv = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [5]:
def get_answer_with_image_latent(question, image_latent):
    model = QuestionInference.QuestionsGen()
    posterior_dist = model.posterior(
        num_traces=100,
        inference_engine = InferenceEngine.IMPORTANCE_SAMPLING,
        observe = {'observed_question': QuestionInference.text_to_vec(question)},
        initial_trace=None
    )
    question_latent = posterior_dist.sample()["question_latent"]
    
    return SymbolicSolver.solve(question_latent, image_latent)

In [8]:
# set up an example image latent
ids = [1, 2, 3, 4, 5, 6]
Xs = [38, 21, 57, 35, 68, 8]
Ys = [52, 17, 49, 40, 63, 55]
color_names = ["red", "green", "blue", "orange", "gray", "yellow"]
shapes = ["circle", "rectangle", "circle", "rectangle", "circle", "rectangle"]
image_latent = pd.DataFrame({"ID": ids, "X": Xs, "Y": Ys, "Color": color_names, "Shape": shapes})

In [9]:
image_gen = ImageGenerator.ImagesGen()

In [10]:
image_gen.forward()

[(0, tensor([51,  9]), 'r'),
 (1, tensor([18, 57]), 'c'),
 (2, tensor([59, 24]), 'r'),
 (3, tensor([38, 68]), 'c'),
 (4, tensor([40, 12]), 'c'),
 (5, tensor([23, 44]), 'c')]

In [17]:
def compile_image_latent(objects):
    IDs = []
    Xs = []
    Ys = []
    colors = []
    shapes = []
    for i in range(len(objects)):
        obj = objects[i]
        IDs.append(i)
        Xs.append(obj[1][0].item())
        Ys.append(obj[1][1].item())
        colors.append(color_names[i])
        shapes.append(obj[2])
    return pd.DataFrame({"ID": IDs, "X": Xs, "Y": Ys, "Color": colors, "Shape": shapes})

In [12]:
get_answer_with_image_latent("Which shape is closest to the blue object?", image_latent)

Time spent  | Time remain.| Progress             | Trace   | ESS   | Traces/sec
0d:00:00:00 | 0d:00:00:00 | #################### | 100/100 |  1.00 | 718.34       


'circle'

## Unseen Question Analysis

In [13]:
image_latent

,ID,X,Y,Color,Shape,distance
0,1,38,52,red,circle,3.610000e+02
1,2,21,17,green,rectangle,1.296000e+03
2,3,57,49,blue,circle,9.000000e+10
3,4,35,40,orange,rectangle,4.840000e+02
4,5,68,63,gray,circle,1.210000e+02
5,6,8,55,yellow,rectangle,2.401000e+03


In [14]:
new_questions = ("What colour is the object closest to the {} object?",
                "Is the {} object on the right?",
                "Is the {} object on the bottom?",
                "What shape is the most common?",
                "How many objects are on the left side of the screen?",
                "How many objects are on the top of the screen?")

In [15]:
def solve_new_question(i, image_latent, color="red"):
    if i == 0:
        main_object = None
        for index, row in image_latent.iterrows():
            if row["Color"] == color:
                main_object = row
                break
        image_latent["distance"] = image_latent.apply(lambda x: (x["X"] - main_object["X"])**2 + (x["Y"] - x["Y"])**2 if x["Color"] != main_object["Color"] else -9e10, axis=1)
        farthest_object = image_latent.loc[image_latent["distance"].idxmax()]
        return farthest_object["Color"]
    elif i == 1:
        main_object = None
        for index, row in image_latent.iterrows():
            if row["Color"] == color:
                main_object = row
                break
        return int(main_object["X"] > image_size/2)
    elif i == 2:
        main_object = None
        for index, row in image_latent.iterrows():
            if row["Color"] == color:
                main_object = row
                break
        return int(main_object["Y"] > image_size/2)
    elif i == 3:
        count_rectangle = 0
        count_circle = 0
        for index, row in image_latent.iterrows():
            if row["Shape"] == "circle":
                count_circle += 1
            else:
                count_rectangle += 1
        if count_circle >= count_rectangle:
            return "circle"
        else:
            return "rectangle"
    elif i == 4:
        count_left = 0
        for index, row in image_latent.iterrows():
            if row["X"] < image_size/2:
                count_left += 1
        return count_left
    elif i == 3:
        for index, row in image_latent.iterrows():
            if row["Y"] < image_size/2:
                count_top += 1
        return count_top

In [21]:
from collections import defaultdict

In [ ]:
image_size = 75
all_n_correct = []
all_n_total = []
n_correct_by_q = defaultdict(int)
n_total_by_q = defaultdict(int)
for img in range(1000):
    n_correct = 0
    n_total = 0
    image_latent = compile_image_latent(image_gen.forward())
    for i in range(len(new_questions)):
        question = new_questions[i]
        if i < 3:
            for color in color_names:
                question = question.format(color)
                true_answer = solve_new_question(i, image_latent, color)
                inferred_answer = get_answer_with_image_latent(question, image_latent)
                if true_answer == inferred_answer:
                    n_correct += 1
                    n_correct_by_q[i] += 1
                n_total += 1
                n_total_by_q[i] += 1
        else:
            true_answer = solve_new_question(i, image_latent)
            inferred_answer = get_answer_with_image_latent(question, image_latent)
            if true_answer == inferred_answer:
                n_correct += 1
                n_correct_by_q[i] += 1
            n_total += 1
            n_total_by_q[i] += 1
    all_n_correct.append(n_correct)
    all_n_total.append(n_total)

Time spent  | Time remain.| Progress             | Trace   | ESS   | Traces/sec
0d:00:00:00 | 0d:00:00:00 | #################### | 100/100 |  4.00 | 808.60       
Time spent  | Time remain.| Progress             | Trace   | ESS   | Traces/sec
0d:00:00:00 | 0d:00:00:00 | #################### | 100/100 |  2.00 | 808.86       
Time spent  | Time remain.| Progress             | Trace   | ESS   | Traces/sec
0d:00:00:00 | 0d:00:00:00 | #################### | 100/100 |  4.00 | 928.11       
Time spent  | Time remain.| Progress             | Trace   | ESS   | Traces/sec
0d:00:00:00 | 0d:00:00:00 | #################### | 100/100 |  5.00 | 895.24       
Time spent  | Time remain.| Progress             | Trace   | ESS   | Traces/sec
0d:00:00:00 | 0d:00:00:00 | #################### | 100/100 |  3.00 | 887.05       
Time spent  | Time remain.| Progress             | Trace   | ESS   | Traces/sec
0d:00:00:00 | 0d:00:00:00 | #################### | 100/100 |  3.00 | 887.86       
Time spent  | Time rem

In [31]:
np.mean(all_n_correct)

5.9

In [32]:
np.mean(all_n_total)

21.0

In [34]:
n_correct_by_q

defaultdict(int, {1: 285, 2: 283, 4: 22})

In [35]:
n_total_by_q

defaultdict(int, {0: 600, 1: 600, 2: 600, 3: 100, 4: 100, 5: 100})